* Dates:
    * Older moves might not be torrented
    * Month: Blockbusters are released in May and December, No good movies released January - April, Kids are on summer vaction during summer months.
* Production Budget: 
    * Might be outliers. Take log if outliers, Remove outliers, Create new feature that is binning production budget
* Rating: Is there a relationship with Number of torrents, look at mean, median torrents per rating. Maybe group ratings together for example G, PG, PG-13 in one group R in another
* Genre as a feature. Maybe keep a subset maybe create new ratings by combing other ratings

In [259]:
import pandas as pd
import numpy as np

import itertools
import operator
import string
from collections import defaultdict

In [260]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF

# plotly username and password kept in separate file
plot_df = pd.read_csv('data/plotly.csv')
plotly.tools.set_credentials_file(username=plot_df.columns[0], api_key=plot_df.columns[1])
text_color = 'rgb(107, 107, 107)'

# Read in data and convert dates to datetime objects

In [261]:
# csv into pandas dataframe
df = pd.read_csv('data/data.csv', encoding='latin-1')

# update dates to datetime objects
df['Released'] = pd.to_datetime(df['Released'])
df['Year'] = pd.DatetimeIndex(df['Released']).year
df['Year_Int'] = pd.to_numeric(df['Year'])
df['Month'] = pd.DatetimeIndex(df['Released']).month

# Look at Number of Titles Released per Year in Dataset

In [262]:
# number of titles per year in dataset
df_yr = df['Year'].value_counts().reset_index()
df_yr.columns = ['Year','Count']

In [263]:
# bar chart of titles per year in dataset
trace = go.Bar(x=df_yr['Year'], y=df_yr['Count'], marker=dict(color=colors[4]))

data = [trace]
layout = go.Layout(
    title='Movies Released per Year',
    xaxis=dict(
        title='Release Year',
        tickfont=dict(size=14, color=text_color)),
    yaxis=dict(
        title='Number of Movies',
        titlefont=dict(size=16, color=text_color),
        tickfont=dict(size=14, color=text_color)),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='style-bar')

# Trim Dataset by Years of Interest

In [264]:
# trim dataframe to "relevant" years
def dataframe_trim(start_incl, stop_incl, feature, dataframe):
    mask = (dataframe[feature] >= start_incl) & (dataframe[feature] <= stop_incl)
    dataframe = dataframe.loc[mask]
    return dataframe

before = len(df)
yr_cut_bot = 1998
yr_cut_top = 2015

df = dataframe_trim(yr_cut_bot, yr_cut_top, 'Year', df)

after = len(df)
print('{0} entries lost ({1}%) due to date cutoff between {2} and {3}'.format(before-after, 
                                                round((before/after)/before *100, 2), yr_cut_bot, yr_cut_top))

370 entries lost (0.08%) due to date cutoff between 1998 and 2015


In [265]:
# number of titles per year in dataset 
df_yr = df['Year'].value_counts().reset_index()
df_yr.columns = ['Year','Count']

In [266]:
# bar chart of titles per year in dataset
trace = go.Bar(x=df_yr['Year'], y=df_yr['Count'], marker=dict(color=colors[4]))

data = [trace]
layout = go.Layout(
    title='Movies Released per Year ({0}-{1})'.format(yr_cut_bot, yr_cut_top),
    xaxis=dict(
        title='Release Year',
        tickfont=dict(size=14,color=text_color)),
    yaxis=dict(
        title='Number of Movies',
        titlefont=dict(size=16, color=text_color),
        tickfont=dict(size=14, color=text_color)),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='style-bar')

In [267]:
_ = '''
# 1998
mask =  (df_yr['Year'] == 1998) & (df_yr['Total_Torrents'] > 100)
df_yr = df_yr[~mask]

# 1999
mask =  (df_yr['Year'] == 1999) & (df_yr['Total_Torrents'] > 200)
df_yr = df_yr[~mask]

# 2000
mask =  (df_yr['Year'] == 2000) & (df_yr['Total_Torrents'] > 200)
df_yr = df_yr[~mask]

# 2001
mask =  (df_yr['Year'] == 2001) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2002
mask =  (df_yr['Year'] == 2002) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2003
mask =  (df_yr['Year'] == 2003) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2004
# 2005
mask =  (df_yr['Year'] == 2005) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2006
mask =  (df_yr['Year'] == 2006) & (df_yr['Total_Torrents'] > 300)
df_yr = df_yr[~mask]

# 2007
mask =  (df_yr['Year'] == 2007) & (df_yr['Total_Torrents'] > 450)
df_yr = df_yr[~mask]

# 2010
mask =  (df_yr['Year'] == 2010) & (df_yr['Total_Torrents'] > 800)
df_yr = df_yr[~mask]

# 2014
mask =  (df_yr['Year'] == 2014) & (df_yr['Total_Torrents'] > 850)
df_yr = df_yr[~mask]
'''

In [268]:
# convert genres string to list of genres
unique_genres = df['Genre'].unique()
unique_genres = unique_genres.tolist()

lst_grouped_genres = [[x.strip() for x in lst.split(',')] for lst in unique_genres]

In [269]:
# list of individual genres in dataset
individual_genres = list(set(itertools.chain(*lst_grouped_genres)))

In [270]:
# dictionary of genre counts
genre_count_dict = defaultdict(lambda:0)
for genre in individual_genres:
    genre_count_dict[genre] = df['Genre'].str.contains(genre).sum()

# sort genre dictionary by year
genre_count_lst = sorted(genre_count_dict.items(), key=operator.itemgetter(1))
genre_count_lst = genre_count_lst[::-1]

# list of genres and list of genre counts
genre, genre_count = zip(*genre_count_lst)

In [271]:
# bar chart of each genre in dataset
trace = go.Bar(x=genre, y=genre_count, marker=dict(color=colors[5]))

data = [trace]
layout = go.Layout(
    title='Genre Occurences ({0}-{1})'.format(yr_cut_bot, yr_cut_top),
    xaxis=dict(
        title='Genre',
        tickfont=dict(size=14, color=text_color)),
    yaxis=dict(
        title='Occurences in Classification',
        titlefont=dict(size=16, color=text_color),
        tickfont=dict(size=14, color=text_color)),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='style-bar')

In [272]:
# split genre strings into a numpy array
def split_to_array(ser):
    split_array = np.array(ser.strip().replace(',','').split(' '))
    return pd.Series(split_array)

# turn numpy array into count of genre occurances
genres = df['Genre'].apply(split_to_array)
genres = pd.Series(genres.values.ravel()).dropna()
genres = genres.value_counts().sort_values(ascending=False)

In [273]:
# return genre that occurs the most (in total data set) in the list of genres (for that row)
def convert_frequency(ser, genres=genres):
    split_array = np.array(ser.strip().replace(',','').split(' '))
    genre = genres.loc[split_array].argmax()
    return genre

# add new column to dataframe classifying genre list as single genre of significance
df['Genre_Single'] = df['Genre'].apply(convert_frequency)

In [284]:
# look at number of single genre counts after extraction
df_count = df_sub['Genre_Single'].value_counts().reset_index()
df_count.columns = ['Genre_Single', 'Count']

In [287]:
# bar chart of significant single genre in dataset
trace = go.Bar(x=df_count['Genre_Single'], y=df_count['Count'], marker=dict(color=colors[5]))

data = [trace]
layout = go.Layout(
    title='Single Genre Classification Count ({0}-{1})'.format(yr_cut_bot, yr_cut_top),
    xaxis=dict(
        title='Genre',
        tickfont=dict(size=14, color=text_color)),
    yaxis=dict(
        title='Occurences in Classification',
        titlefont=dict(size=16, color=text_color),
        tickfont=dict(size=14, color=text_color)),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='style-bar')

# Limit Dataset to Columns of Interest

In [306]:
df = df[['Prod_Budget', 'Year', 'Month', 'Runtime', 'Total_Torrents', 'Genre_Single']]

In [307]:
colors = ['rgb(189, 195, 199)', 'rgb(54, 215, 183)', 'rgb(154, 18, 179)', 'rgb(174, 168, 211)', 
          'rgb(31, 58, 147)', 'rgb(142, 68, 173)','rgb(25, 181, 254)','rgb(46, 204, 113)', 
          'rgb(241, 196, 15)', 'rgb(249, 105, 14)', 'rgb(242, 38, 19)']

In [308]:
def make_bar_trace(x_years, y_counts, name_genre, i):
    trace = go.Bar(
        x = x_years, 
        y = y_counts, 
        name = name_genre, 
        marker=dict(
            color=colors[i]),
        opacity=0.8
        )
    return trace

In [309]:
years = df['Year'].sort_values().unique()
individual_genres = df['Genre_Single'].unique()

In [310]:
# dictionary of genre counts by year
genre_count_dict = defaultdict(lambda:0)
for genre in individual_genres:
    genre_count_dict[genre] = df['Genre_Single'].str.contains(genre).sum()

# sort genre dictionary by year
genre_count_lst = sorted(genre_count_dict.items(), key=operator.itemgetter(1))
genre_count_lst = genre_count_lst[::-1]

# list of genres and list of genre counts
genre, genre_count = zip(*genre_count_lst)

(533, 389, 213, 57, 52, 27, 23)

In [104]:
traces = []

for i,genre in enumerate(ordered_select_genre[::-1]):
    year_lst = []
    count_lst = []
    year_lst = [x for x in genre_dict[genre].keys()]
    count_lst = [x for x in genre_dict[genre].values()]
    
    traces.append(make_bar_trace(year_lst, count_lst, genre, i))

data = traces

layout = go.Layout(
    title='Movie Titles ({0}-{1})'.format(yr_cut_bot, yr_cut_top),
    xaxis=dict(
        title='Year',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Number of Titles',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    barmode='stack',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

In [18]:
def make_filled_line_trace(x_years, y_counts, name_genre, color):
    trace = go.Scatter(
        x = x_years, 
        y = y_counts, 
        name = name_genre,
        mode = 'lines',
        line = dict(width=0.5,
                   color = color),
        fill = 'tonexty')
    return trace

# need to get total torrent counts for that year to divide by for percentage

In [90]:
new_dict = {}
for genre,info in genre_dict.items():
    
    for yr,ct in v.items():
        

In [66]:
traces = []
sum_lst = np.linspace(0,0,len(df['Year'].unique()))

for i, genre in enumerate(ordered_select_genre[::-1]):
    count_lst = []
    for i, yr_ct in enumerate(genre_dict[genre].items()):
        if yr_ct[0] == year_lst[i]:
            count_lst.append(yr_ct)
        #else:
         #   count_lst.append((year_lst[i],0))
    traces.append(count_lst)
    print(len(count_lst))
    #count_lst = [100*x/sum(genre_dict[genre].values()) for x in genre_dict[genre].values()]
    #sum_lst = np.asarray(count_lst) +  sum_lst
    #traces.append((year_lst, sum_lst, genre, colors[i]))

21
2
21
21
21
21
21
21
21
21
21


In [22]:
traces = []

for i, genre in enumerate(ordered_select_genre[::-1]):
    year_lst = []
    count_lst = []
    
    year_lst = [x for x in genre_dict[genre].keys()]
    count_lst = [100*x/sum(genre_dict[genre].values()) for x in genre_dict[genre].values()]
    traces.append(make_filled_line_trace(year_lst, count_lst, genre, colors[i]))

data = traces

In [23]:
import plotly.plotly as py
import plotly.graph_objs as go

data = traces
layout = go.Layout(
    showlegend=True,
    xaxis=dict(
        type='category',
    ),
    yaxis=dict(
        type='linear',
        range=[1, 100],
        dtick=20,
        ticksuffix='%'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-area-plot')


# Select only significant values from dataframe

In [17]:
# trim dataframe to only genres of significance
df = df.loc[df['Genre_Single'].isin(ordered_select_genre)]

# select only genres of significance
ratings = ['PG-13', 'PG', 'G', 'R']
df_sub = df_sub.loc[df_sub['Rated'].isin(ratings)]

#df_sub['Runtime'].value_counts()
#df_sub['Genre_Single'].value_counts()
#df_sub['Rated'].value_counts()
df_sub.describe()

NameError: name 'df_yr' is not defined

In [ ]:
# entire dataframe
plt.rcParams['figure.figsize'] = (15, 15)
_ = pd.tools.plotting.scatter_matrix(df_sub)

In [ ]:
import plotly.plotly as py
from plotly.tools import FigureFactory as FF

df_a = df_yr[['Prod_Budget', 'Year', 'Month', 'Total_Torrents']]
fig = FF.create_scatterplotmatrix(df_a, diag='histogram', index='Month',
                                  height=800, width=800)
py.iplot(fig, filename='Histograms along Diagonal Subplots')

In [ ]:
from patsy import dmatrices
patsy_formula = 'Total_Torrents ~ Prod_Budget + Year + Month + Runtime + Genre_Single'
y, x = dmatrices(patsy_formula, data=df_sub, return_type='dataframe')

In [ ]:
import statsmodels.api as sm
model = sm.OLS(y, x)
results = model.fit()
results.summary()

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, y)
mod_lr_score = model.score(x, y)
mod_lr_coef = model.coef_
model.results

In [ ]:
from sklearn import cross_validation as cv
from sklearn import metrics

x_train, x_test, y_train, y_test = cv.train_test_split(x,y,test_size=0.20,random_state=1234)

model = LinearRegression().fit(x_train, y_train)

# store results
mean_sq_err = metrics.mean_squared_error(y_train,model.predict(x_train))
cv_mod_score = model.score(x_train, y_train)

In [ ]:
# reset x, y otherwise errors occur
y, x = dmatrices(patsy_formula, data=df_sub, return_type='dataframe')

from sklearn.cross_validation import KFold
kf = KFold(len(df_sub), n_folds=10, shuffle=True)

for train_index, test_index in kf:
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf2 = LinearRegression().fit(x.iloc[train_index], y.iloc[train_index])

# store results
mean_sq_errKf = metrics.mean_squared_error(y_train,model.predict(x_train))
cvKf_mod_score = clf2.score(x,y)

In [ ]:
#NORMAL RESULTS
print('Model Linear Regression Score = {0}'.format(mod_lr_score))
print('            Mean Square Error = {0}'.format(mean_sq_err))
print(' Cross Validation Model Score = {0}'.format(cv_mod_score))
print('     Mean Squred Error K-Fold = {0}'.format(mean_sq_errKf))
print('Cross Val. K-Fold Model Score = {0}'.format(cvKf_mod_score))

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(results,'Prod_Budget', fig=fig)

---
# Log Transform

In [ ]:
df.columns

In [ ]:
df_sub['log_budg']=np.log(df_sub.Prod_Budget)
#df_sub['log_year']=np.log(df_sub.Year)
#df_sub['log_run']=np.log(df_sub.Runtime)
df_sub['log_tor']=np.log(df_sub.Total_Torrents)

trans = df_sub[['log_budg', 'Year', 'log_tor']]
plt.rcParams['figure.figsize'] = (15, 15)
pd.tools.plotting.scatter_matrix(trans)

In [ ]:
log_patsy_formula = 'log_tor ~ log_budg + Year + Month'
y, x = dmatrices(log_patsy_formula, data=df_sub, return_type='dataframe')

In [ ]:
import plotly.plotly as py
from plotly.tools import FigureFactory as FF

df_a = df_sub[['log_budg', 'Year', 'Month', 'log_tor']]
fig = FF.create_scatterplotmatrix(df_a, diag='histogram', index='Month',
                                  height=800, width=800)
py.iplot(fig, filename='Histograms along Diagonal Subplots')

In [ ]:
import statsmodels.formula.api as smf
results = smf.ols(formula=log_patsy_formula, data=df_sub,).fit()
results.summary()

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, y)

# store results
log_mod_lr_score = model.score(x,y)

In [ ]:
from sklearn import cross_validation as cv
from sklearn import metrics

x_train, x_test, y_train, y_test = cv.train_test_split(x,y,test_size=0.20,random_state=1234)

model = LinearRegression().fit(x_train, y_train)

# store results
log_mean_sq_err = metrics.mean_squared_error(y_train,model.predict(x_train))
log_cv_mod_score = model.score(x_train, y_train)

In [ ]:
# reset x, y otherwise errors occur
y, x = dmatrices(log_patsy_formula, data=df_sub, return_type='dataframe')

from sklearn.cross_validation import KFold
kf = KFold(len(df_sub), n_folds=10, shuffle=True)

for train_index, test_index in kf:
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf2 = LinearRegression().fit(x.iloc[train_index], y.iloc[train_index])

# store results
log_mean_sq_errKf = metrics.mean_squared_error(y_train,model.predict(x_train))
log_cvKf_mod_score = clf2.score(x,y)

In [ ]:
#LOG RESULTS
print('Log Model Linear Regression Score = {0}'.format(log_mod_lr_score))
print('            Log Mean Square Error = {0}'.format(log_mean_sq_err))
print(' Log Cross Validation Model Score = {0}'.format(log_cv_mod_score))
print('     Log Mean Squred Error K-Fold = {0}'.format(log_mean_sq_errKf))
print('Log Cross Val. K-Fold Model Score = {0}'.format(log_cvKf_mod_score))

In [ ]:
df_TEST = pd.read_csv('data/test_data2.csv', encoding='latin-1')

df_TEST['log_budg']=np.log(df_TEST.Prod_Budget)
df_TEST['log_run']=np.log(df_TEST.Runtime)
df_TEST['log_tor']=np.log(df_TEST.Total_Torrents)

def split_to_array(ser):
    split_array = np.array(ser.strip().replace(',','').split(' '))
    return pd.Series(split_array)

genres = df_yr.Genre.apply(split_to_array)
genres = pd.Series(genres.values.ravel()).dropna()
genres = genres.value_counts().sort_values(ascending=False)

def convert_frequency(ser, genres=genres):
    split_array = np.array(ser.strip().replace(',','').split(' '))
    genre = genres.loc[split_array].argmax()
    return genre

df_TEST['Genre_Single'] = df_TEST.Genre.apply(convert_frequency)

log_patsy_formula_test = 'log_tor ~ log_budg + Year + Month + log_run + Genre_Single'
y, x = dmatrices(log_patsy_formula_test, data=df_TEST, return_type='dataframe')

print(clf2.score(x_test, y_test))
print(metrics.mean_squared_error(y_test,model.predict(x_test)))

In [ ]:
_ = plt.plot(y, model.predict(x), 'bo')

In [ ]:
plt.figure(figsize=(25,10))

ind = np.arange(len(yr_dict))
width = 0.35

bar_year = [year for year, count in yr_lst]
bar_count = [count for year, count in yr_lst]

plt.bar(ind, bar_count, width, color='r')

plt.ylabel('Count')
plt.xlabel('Year')
plt.title('Number of Torrents per Year')
plt.xticks(ind + width/2., (bar_year), rotation='vertical')
plt.yticks(np.arange(0, 91, 5))

plt.show()

In [ ]:
#log_tor ~ log_budg + Year + Month + log_run + Genre_Single'

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(results,'log_budg', fig=fig)

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(results,'Year', fig=fig)

In [ ]:
fig = plt.figure(figsize=(12,8))
fig = sm.graphics.plot_regress_exog(results,'Month', fig=fig)